In [3]:
import pandas as pd 
from tqdm import tqdm
import numpy as np
from IPython.display import display

In [11]:
# Load the csv
input_csv = '/home/jovyan/ODBiz/4-Parsing/output/2-parsed_biz.csv'
input_csv = '/home/jovyan/ODBiz/4-Parsing/output/parsed_biz.csv'
total_lines = 803658
chunksize = 10000
df = pd.concat([chunk for chunk in tqdm(pd.read_csv(input_csv,
                                        chunksize=chunksize, 
                                        dtype=str), 
                                    desc='Loading data', 
                                    total=total_lines//chunksize+1)
                ])
num_of_rows = df.shape[0]
print(f'Successfully loaded {input_csv}')
print(f'df has {num_of_rows} rows')

localfiles = pd.unique(df['localfile'])


Loading data: 100%|██████████| 81/81 [00:06<00:00, 13.01it/s]
Successfully loaded /home/jovyan/ODBiz/4-Parsing/output/parsed_biz.csv
df has 803658 rows


In [12]:
df2 = df.set_index('idx')
x = df2[df2.index.duplicated()]
print(f'{x.shape[0]} rows found')


74 rows found


In [2]:
# Check entries to see if LP parsed them correctly after removing spaces adjacent to dashes
new_df = df.copy()
new_df = new_df.set_index('idx')

txt_idxs_file = 'incorrectly_parsed_idxs.txt'
with open(txt_idxs_file) as fp:
    idxs = fp.readlines()
    idxs = [line.rstrip() for line in idxs]

# idxs = ''

# result_df = new_df.loc[idxs].copy()
# rows = result_df.shape[0]
# print(f'Found {rows} results')

pd.options.display.max_columns = None
drows = 100
pd.options.display.max_rows = drows
display(new_df.loc[idxs, ['localfile', 'business_name', 'full_address', 
'LP_Unit', 'LP_street_no', 'LP2_unit', 'LP2_street_no', 'LP_street_name', 'postal_code',
]])


NameError: name 'df' is not defined

Index(['idx', 'localfile', 'business_name', 'business_sector',
       'business_subsector', 'business_description', 'business_id_no',
       'licence_number', 'licence_type', 'primary_NAICS', 'secondary_NAICS',
       'NAICS_descr', 'alt_econ_act_code', 'alt_econ_act_descrip', 'latitude',
       'longitude', 'full_address', 'full_address_2', 'mailing_address',
       'unit', 'street_no', 'street_name', 'street_direction', 'street_type',
       'city', 'province', 'postal_code', 'country', 'business_website',
       'email', 'telephone', 'telephone_extension', 'toll_free_telephone',
       'fax', 'total_no_employees', 'no_full_time', 'no_part_time',
       'no_seasonal', 'date_established', 'indigenous', 'status', 'provider',
       'geo_source', 'LP_street_no', 'LP_street_name', 'LP_City',
       'LP_Province', 'LP_PostCode', 'LP_Unit', 'LP2_unit', 'LP2_street_no',
       'spill', 'LP3_unit'],
      dtype='object')

In [6]:
# Test out pattern searching
#	<unit>\s-\s<street_no>\s<street_name>

re_pat = r'^[\dA-z#]*\s+-\s?|^[\dA-z#]*\s?-\s+'
# re_pat = r'-'
new_df = df.copy()
idxs = (~new_df['full_address'].str.contains(re_pat, na = False)) & (new_df['full_address'].str.contains(r'\s+-\s?|\s?-\s+', na = False)) | (new_df['LP_street_no'].str.contains(' ', na = False))
idxs = (new_df['full_address'].str.contains(r'\s+-\s?|\s?-\s+', na = False)) #| (new_df['LP_street_no'].str.contains(' ', na = False))

result_df = new_df.loc[idxs].copy()
rows = result_df.shape[0]
print(f'Found {rows} results')

pd.options.display.max_columns = None
drows = 100
pd.options.display.max_rows = drows
display(result_df[['idx', 'localfile', 'business_name', 'full_address', 
'LP_Unit', 'LP_street_no', 'LP2_unit', 'LP2_street_no', 'LP_street_name', 'postal_code',
]])

out_name = '/home/jovyan/ODBiz/4-Parsing/double_check/dashes_with_spaces.csv'
result_df.to_csv(out_name)
print(f'Saved to {out_name}')

Found 29257 results


,idx,localfile,business_name,full_address,LP_Unit,LP_street_no,LP2_unit,LP2_street_no,LP_street_name,postal_code
29508,195031ef907dc24ae947,BC_Indigenous_Business_Listings.csv,A. J. Towing Inc,101 - 45635 McIntosh Dr,NaN,101,NaN,101,mcintosh dr,V2P6V4
29603,5870fb0ddc6b4d0542f3,BC_Indigenous_Business_Listings.csv,Best Bet Entertainment,60B - 966 SW Marine Drive,NaN,60b 966,NaN,60b 966,sw marine drive,V6P5Z2
29619,badd054782efa2b6f4cb,BC_Indigenous_Business_Listings.csv,BladeRunners,Bldg 2C - 7201 Vedder Road,bldg 2c,7201,bldg 2c,7201,vedder road,V2R4G5
29641,03d3462bcdb0affa5e4c,BC_Indigenous_Business_Listings.csv,Buffalo Rains Network Corporation,Suite 115 - 887 Great Northern Way,suite 115,887,suite 115,887,great northern way,V5T4T5
29680,3c3cf2a8ca41fd958049,BC_Indigenous_Business_Listings.csv,CEEM Earth Environmental Management,203 - 175 - 4th Ave,NaN,203 175,NaN,203 175,4th ave,V2C3N3
...,...,...,...,...,...,...,...,...,...,...
797573,fa35d3f3cc7751e5bfb1,Indigenous_Business_Directory.csv,Windtalker Energy Services Inc.,450 630 - 6th Ave SW,NaN,450 630,NaN,450 630,6th ave sw,T2P0S8
802431,49ab726034fb4d8e19a0,ON_Ottawa_Cultural_Businesses.csv,XYZ RGB,4 - 155 Terence Matthews Cresent,NaN,4 155,NaN,4 155,terence matthews cresent,K2M2A8
802465,d803cea2144405b1f575,ON_Ottawa_Cultural_Businesses.csv,Fuel Industries,100 - 7 Hinton Avenue,NaN,100 7,NaN,100 7,hinton avenue,K1Y4P1
803021,35c327d94d0251bc8c11,ON_Welland_Business_Directory.csv,"Burch, Jeff - MPP Constituency Office",102- 60 King Street,NaN,102 60,NaN,102 60,king street,L3B6A4


Saved to /home/jovyan/ODBiz/4-Parsing/double_check/dashes_with_spaces.csv


0         False
1         False
2         False
3         False
4         False
          ...  
803653    False
803654    False
803655    False
803656    False
803657    False
Name: full_address, Length: 803658, dtype: bool

In [ ]:
# # Extract only entries that spillover their unit+street_no values
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
# new_df = df[~df['spill'].isna()]
# new_df = new_df[['idx', 'localfile', 'business_name', 'full_address', 'LP2_unit', 'LP2_street_no', 'spill', 'LP_street_no', 'LP_street_name', 'LP_City', 'LP_Province', 'LP_PostCode', 'LP_Unit', 'LP3_unit']]

# # Replace dashes to avoid Excel's date conversion
# new_df['LP2_unit'] = new_df['LP2_unit'].str.replace('-', '--')
# new_df['LP_street_no'] = new_df['LP_street_no'].str.replace('-', '--')

# # Save to csv
# new_df_path = '/home/jovyan/ODBiz/4-Parsing/double_check/2-parsed_with_spillover.csv'
# new_df.to_csv(new_df_path, index = False)
# print(f'Saved new_df to {new_df_path}')
# display(new_df)

In [18]:
def sequence():
    for i in range(60):
        yield i

seq_gen = sequence()

In [38]:
localfiles

['BC_Victoria_Business_Licences.csv',
 'BC_Chilliwack_Business_Licences.csv',
 'ON_Brampton_Business_Directory.csv']

In [51]:
# Select one localfile at a time to sample
# i = next(seq_gen)
i = 23
print("i =", i)
localfile = localfiles[i]
print("localfile =", localfile)
new_df = df[(df['localfile'] == localfile) & (~df['full_address'].isna())] 


# Extract only entries that spillover their unit+street_no values
pd.options.display.max_columns = None
mrows = 100
pd.options.display.max_rows = mrows
new_df = new_df[[
    'idx', 'localfile', 'business_name', 'full_address', 'LP2_unit', 'LP2_street_no', 'LP_street_name', 'postal_code',
    #  'spill', 'LP_street_no', 'LP_City', 'LP_Province', 'LP_PostCode', 'LP_Unit', 'LP3_unit'
    ]]

# Replace dashes to avoid Excel's date conversion
new_df['LP2_unit'] = new_df['LP2_unit'].str.replace('-', '--')
# new_df['LP_street_no'] = new_df['LP_street_no'].str.replace('-', '--')

display(new_df[:mrows])

i = 23
localfile = QC_Etablissements.csv


,idx,localfile,business_name,full_address,LP2_unit,LP2_street_no,LP_street_name,postal_code
252955,709874226b83eadd1534,QC_Etablissements.csv,LES GRAPHOIDES,362-247 rue Saint-Vallier E,362,247,rue saint-vallier e,G1K3P4
252956,11f8a04dc285c9a3fabb,QC_Etablissements.csv,9000-0084 QUÉBEC INC.,3711 boul. Harvey,NaN,3711,boul. harvey,G7X3A7
252957,6212129e3f30dfd9777d,QC_Etablissements.csv,LE SHACK DU MONT-TREMBLANT,"3035, CHEMIN PRINCIPAL C.P. 405",NaN,c.p. 405,chemin principal,J0T1Z0
252958,d8673c1f6fdee0cbffcd,QC_Etablissements.csv,INFAX,2962 rue de la Promenade,NaN,2962,rue de la promenade,G1W2J6
252959,c0b4bc8fe04ae29b21c5,QC_Etablissements.csv,TRANSPORT CLÉMENT VIAU,1691 ch. Principal,NaN,1691,ch. principal,J0N1M0
252960,f269811dec9f7452d11d,QC_Etablissements.csv,RÊVES ET JARDINS,1080 ch. de Knowlton,NaN,1080,ch. de knowlton,J0E2P0
252961,ad6b99f6f27f3394a550,QC_Etablissements.csv,9000-1488 QUÉBEC INC.,117-2555 av. Dollard,117,2555,av. dollard,H8N3A9
252962,22345840b73b34a12d64,QC_Etablissements.csv,QUALIVIN INC.,9-5645 Grande Allée,9,5645,grande allée,J4Z3G3
252963,3a74185909feaae2f664,QC_Etablissements.csv,Duvaltex (Canada) inc.,662 9e Avenue,NaN,662,9e avenue,G5X1C6
252964,41ce77cc09e9724fd9f7,QC_Etablissements.csv,Duvaltex (Canada) inc.,2805 90e Rue,NaN,2805,90e rue,G6A1K1


In [42]:
# Print out all affected datasets
arr = pd.unique(new_df['localfile'])
for i in arr:
    print(i)

In [36]:
df2_path = '/home/jovyan/ODBiz/4-Parsing/double_check/parsed_with_spillover2.csv'

# Apply the easiest blanket rule on the specific datasets below
localfiles = [  'BC_Victoria_Business_Licences.csv',
                # 'BC_Indigenous_Business_Listings.csv',
                'BC_Chilliwack_Business_Licences.csv',
                'ON_Brampton_Business_Directory.csv',
                # 'QC_Etablissements.csv',
                # 'ON_Toronto_Business_Licences.csv',
                # 'Indigenous_Business_Directory.csv',
                ]
localfiles_idx = new_df['localfile'].isin(localfiles)
df2 = new_df[localfiles_idx].copy()

# Set right most value as street_no, everything else is unit
df2[['LP2_unit', 'LP2_street_no']] = df2['LP_street_no'].str.rsplit('-', expand = True, n = 1)

# Replace dashes to avoid Excel's date conversion
df2['LP2_unit'] = df2['LP2_unit'].str.replace('-', '--')
df2['LP_street_no'] = df2['LP_street_no'].str.replace('-', '--')

df2.to_csv(df2_path, index = False)
print(f'Saved new_df to {df2_path}')
display(df2)

KeyError: 'LP_street_no'

In [ ]:
dfTO_path = '/home/jovyan/ODBiz/4-Parsing/double_check/parsed_with_spillover_toronto.csv'

# Apply Toronto's modified parsing rule
localfiles = [  
                # 'BC_Victoria_Business_Licences.csv',
                # 'BC_Indigenous_Business_Listings.csv',
                # 'BC_Chilliwack_Business_Licences.csv',
                # 'ON_Brampton_Business_Directory.csv',
                # 'QC_Etablissements.csv',
                'ON_Toronto_Business_Licences.csv',
                # 'Indigenous_Business_Directory.csv',
                ]
localfiles_idx = new_df['localfile'] == 'ON_Toronto_Business_Licences.csv'
dfTO = new_df[localfiles_idx].copy()

has_comma = dfTO['full_address'].str.contains(',')

# If there's no comma, set right most value as street_no, everything else is unit
dfTO.loc[~has_comma, ['LP2_unit', 'LP2_street_no']] = dfTO.loc[~has_comma, 'LP_street_no'].str.rsplit('-', expand = True, n = 1).rename(columns = {0: 'LP2_unit', 1: 'LP2_street_no'})

# If there is a comma, apply parsing rule
dfTO.loc[has_comma, ['LP2_street_no', 'LP2_unit']] = dfTO.loc[has_comma, 'full_address'].str.extract('(\d+)[A-Z\s]*,\s?(.*)').rename(columns = {0: 'LP2_street_no', 1: 'LP2_unit'})

# Replace dashes to avoid Excel's date conversion
dfTO['LP2_unit'] = dfTO['LP2_unit'].str.replace('-', '--')
dfTO['LP_street_no'] = dfTO['LP_street_no'].str.replace('-', '--')

dfTO.to_csv(dfTO_path, index = False)
print(f'Saved new_df to {dfTO_path}')
display(dfTO)

In [ ]:
dfTO.loc[~has_comma, 'LP_street_no'].str.rsplit('-', expand = True, n = 1).rename(columns = {0: 'LP2_unit', 1: 'LP2_street_no'})


In [ ]:
dfTO.loc[~has_comma, 'LP_street_no'].str.rsplit('-', expand = True, n = 1)